In [1]:
# Загрузка и обработка данных
import pandas as pd

# Визуализация данных
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

pd.options.display.max_columns = None
warnings.filterwarnings("ignore")


In [3]:
dates_columns = ['date', 'createdDate', 'changedDate']

## Загрузим информацию о моих доходах

In [4]:
data = pd.read_csv('../data/prepared/incomes.csv', parse_dates=dates_columns)
data.head()

,date,categoryName,payee,comment,createdDate,changedDate,incomeAccountName,income,incomeCurrencyShortTitle
0,2023-03-19,NaN,NaN,NaN,2023-03-19 14:44:05,2023-05-02 07:02:35,Тинькофф,1102.0,RUB
1,2023-03-26,Подарок,NaN,Бабушка скинула денежку,2023-03-26 12:23:24,2023-03-26 09:23:31,Тинькофф,1000.0,RUB
2,2023-03-28,NaN,NaN,NaN,2023-03-28 22:47:16,2023-05-02 07:02:35,Тинькофф,1500.0,RUB
3,2023-03-30,NaN,NaN,NaN,2023-03-31 12:04:59,2023-05-02 07:02:35,Тинькофф,231.0,RUB
4,2023-04-05,NaN,NaN,NaN,2023-04-05 17:21:31,2023-05-02 07:02:35,Тинькофф,32.6,RUB


- Добавим дополнительную информацию о месяце/дне недели транзакции для дальнейших группировок

In [5]:
data['month'] = data['date'].dt.month_name()
data['day_of_a_week'] = data['date'].dt.day_name()

## Рассчитаем базовые статистики по моим доходам

In [6]:
mean = data['income'].mean()
median = data['income'].median()
percentile_20 = data['income'].quantile(q=0.2)
percentile_40 = data['income'].quantile(q=0.4)
percentile_60 = data['income'].quantile(q=0.6)
percentile_80 = data['income'].quantile(q=0.8)
maximun = data['income'].max()
minimun = data['income'].min()
standard_deviation = data['income'].std()

result = f"""
<< Базовая статистика >>

+ Средняя сумма поступления: {mean} руб

+ Медианная сумма поступления: {median} руб
+ 20% моих поступлений меньше, чем {percentile_20} рублей
+ 40% моих поступлений меньше, чем {percentile_40} рублей
+ 60% моих поступлений меньше, чем {percentile_60} рублей
+ 80% моих поступлений меньше, чем {percentile_80} рублей

+ Максимальное поступление: {maximun} руб
+ Минимальное поступление: {minimun} руб

* Стандартное отклонение: {standard_deviation} руб
"""
print(result)


<< Базовая статистика >>

+ Средняя сумма поступления: 5042.1978 руб

+ Медианная сумма поступления: 3000.0 руб
+ 20% моих поступлений меньше, чем 780.0000000000001 рублей
+ 40% моих поступлений меньше, чем 2000.0 рублей
+ 60% моих поступлений меньше, чем 3565.999999999997 рублей
+ 80% моих поступлений меньше, чем 5000.0 рублей

+ Максимальное поступление: 200000.0 руб
+ Минимальное поступление: 25.89 руб

* Стандартное отклонение: 16562.109322439464 руб



Уже тут мы видим, что в среднем, перевод на мою карту составляет около 5000 рублей.

Также, уже здесь заметны аномальные значения, например 200000 рублей я получил как приз за победу на Большой Перемене

## Изучим динамику поступлений с течением времени

In [7]:
plot_data = data.groupby('date', as_index=False) \
    .agg({'income' : 'sum'}) \
    .sort_values('date')

px.line(plot_data, x='date', y='income',
        title='Динамика моих доходов с течением времени')

Здесь мы видим очень характерный и заметный пик 18 декабря. В этот день мне пришел приз с конкурса Большая Перемена.

Поскольку это выброс, стоит посмотреть на данные без него

In [8]:
plot_data = plot_data[plot_data['income'] < 100000].copy()

In [9]:
plot_data['rolling'] = plot_data['income'].rolling(7, center=True).mean()

# Используем px.scatter для добавления линии тренда к исходным данным дохода
fig = px.scatter(plot_data, x='date', y='income', trendline="ols",
                 title='Динамика моих доходов с течением времени (Скользящее среднее)')

# Добавляем линию скользящего среднего отдельно
fig.add_scatter(x=plot_data['date'], y=plot_data['rolling'], mode='lines', name='Скользящее среднее')

fig.show()

In [10]:
plot_data = data.groupby(['date', 'categoryName'], as_index=False) \
    .agg({'income' : 'sum'}) \
    .sort_values('date')

plot_data = plot_data[(plot_data['income'] < 100000)].copy()

px.line(plot_data, x='date', y='income', color='categoryName',
        title='Динамика моих доходов с течением времени по категориям')

Как итог, могу сказать, что мои доходы за год выросли. Это связано с моим трудоустройством и тем, что теперь я получаю зарплату

### Посмотрим на месяцы, когда я получал наибольшее число доходов

In [13]:
data.head()

,date,categoryName,payee,comment,createdDate,changedDate,incomeAccountName,income,incomeCurrencyShortTitle,month,day_of_a_week
0,2023-03-19,NaN,NaN,NaN,2023-03-19 14:44:05,2023-05-02 07:02:35,Тинькофф,1102.0,RUB,March,Sunday
1,2023-03-26,Подарок,NaN,Бабушка скинула денежку,2023-03-26 12:23:24,2023-03-26 09:23:31,Тинькофф,1000.0,RUB,March,Sunday
2,2023-03-28,NaN,NaN,NaN,2023-03-28 22:47:16,2023-05-02 07:02:35,Тинькофф,1500.0,RUB,March,Tuesday
3,2023-03-30,NaN,NaN,NaN,2023-03-31 12:04:59,2023-05-02 07:02:35,Тинькофф,231.0,RUB,March,Thursday
4,2023-04-05,NaN,NaN,NaN,2023-04-05 17:21:31,2023-05-02 07:02:35,Тинькофф,32.6,RUB,April,Wednesday


In [19]:
plot_data = data.groupby(['month', 'categoryName'], as_index=False) \
                .agg({'income':'sum'}) \
                .sort_values(by='income', ascending=True)

px.histogram(data_frame=data, x='month', y='income', color='categoryName')

Ну и отсюда очевидно, что декабрь был самым прибыльным месяцем по сравнению с остальными месяцами из-за получения приза с Большой Перемены. Однако если исключить приз из статистики, то самыми прибыльными месяцами станут декабрь февраль и март

### Посмотрим на дни недели, когда я получал наибольшее число доходов

Ну просто ради интереса)

In [21]:
plot_data = data.groupby(['day_of_a_week', 'categoryName'], as_index=False) \
                .agg({'income':'sum'}) \
                .sort_values(by='income', ascending=True)

px.histogram(data_frame=data, x='day_of_a_week', y='income', color='categoryName')

Итак, в среднем, понедельник становится самым прибыльным днем в году.   
Интересно, что выходные, в среднем, становятся наименее прибыльными днями недели

## Изучим распределение категрорий доходов в общей совокупности

In [11]:
plot_data = data['categoryName'].value_counts().reset_index()

px.bar(data_frame=plot_data, x='categoryName', y='count', 
       title = 'Количество транзакций по категориям')

Количественно, наибольшее число переводов я получил от мамы (72 транзакции). После этого идет зарплата (26 транзакций)

In [12]:
plot_data = data.groupby('categoryName', as_index=False) \
                .agg({'income' : 'sum'})

px.pie(data_frame=plot_data, values='income', names='categoryName',
       title = 'Распределение поступлений по категориям и сумме транзакций')

Здесь мы видим, что, по большей мере, в меня вложились родители (40.3%).    
Что ещё более значимо для меня, за год я самостоятельно заработал 182 000 рублей + 200 000 призовых рублей, что ровно половина от всех поступлений на мой счет.